In [1]:
import pandas as pd
import numpy as np
import datetime

#import xgboost as xgb

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.layers import Dropout
from keras.layers import Activation

import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [60]:
#this is the raw dataset
#data16 = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/016.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
#data16_sample = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/data16_sample.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

df = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/ad data/AntzbBackData10k.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

df=df[["account", 'advertisement','title', "content", "clicksCount", "orderNum", "originalFlag", "likeCount", "publicTime"]]


df = df[pd.notnull(df['title'])]
df = df[pd.notnull(df['content'])]
df = df[pd.notnull(df['advertisement'])]
df = df[pd.notnull(df['likeCount'])]

df["publicTime"] = pd.to_datetime(df["publicTime"])
#df["publicTime"] = dd["publicTime"].dt.to_period('D')
df["clicksCount"] = df["clicksCount"].astype(float)
df["likeCount"] = df["likeCount"].astype(float)
df["orderNum"] = df["orderNum"].astype(float)
df["originalFlag"] = df["originalFlag"].astype(float)

#topicdata = pd.read_csv(r'C:/Users/Junhao/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [62]:
#cutword1 = cutword.head(1000).wordsCut
#cutword3 = cutword.head(1000).title  #i can also try to use only the article titles as bert input 
df["combined_content"] = df.title + df.title + df.title + df.content

cutword2 = df.combined_content.dropna()

#cutword2 = df.head(500).content.dropna() #use 1000 article content from rawdata as bert input
#I need to cut each article to the length of 100 characters because the tokenizer can only deal with a max length of 512
#the computation takes too long if the kept article length is long

cutword3 = []
for i in cutword2:
    dd = i[:512]
    cutword3.append(dd)

In [63]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-chinese'

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
MAX_LEN = 512

def tokenize_sentences(sentences, tokenizer, max_seq_len = 512):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(cutword3, tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)
#attention_masks = torch.tensor(attention_masks)
#attention_masks = attention_masks.tolist()
#attention_masks.to_tensor()

<ipython-input-63-4faa14ba2eef>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm(sentences):


  0%|          | 0/9995 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [64]:
#likeCount = df.head(500).likeCount #use likecount as dependent variable 

likeCount = df.likeCount

#df = df[["likeCount", "combined_content"]].head(500)

df = df[["likeCount", "combined_content"]]

#removeinds = pd.isnull(df).any(1).to_numpy().nonzero()[0]

#likeCount = [i for j, i in enumerate(likeCount) if j not in removeinds] # remove the two likecount where content is empty
#likeCount = [float(i) for i in likeCount]



In [22]:
len(likeCount)

9997

In [65]:
#create training and validation set

type_ids = np.zeros((len(likeCount), 512))

attention_val = attention_masks[-int(0.2 * len(likeCount)):]
input_ids_val = input_ids[-int(0.2 * len(likeCount)):]
type_ids_val = type_ids[-int(0.2 * len(likeCount)):]

attention_train = attention_masks[:-int(0.2 * len(likeCount))]
input_ids_train = input_ids[:-int(0.2 * len(likeCount))]
type_ids_train = type_ids[:-int(0.2 * len(likeCount))]


x_train = [input_ids_train, type_ids_train, attention_train]
x_val = [input_ids_val, type_ids_val, attention_val]

y_val = np.asarray(likeCount[-int(0.2 * len(likeCount)):])
y_train = np.asarray(likeCount[:-int(0.2 * len(likeCount))])

In [67]:
x_train[0][0]

array([  101,   711,   784,   720,  5739,  4909,  4518,  1914,  2168,
        1555,  8024,  3791,  4909,  4518,  1316,  2768,   749,  7484,
        1462,  2157,  4638,  1921,  1828,  8043,   711,   784,   720,
        5739,  4909,  4518,  1914,  2168,  1555,  8024,  3791,  4909,
        4518,  1316,  2768,   749,  7484,  1462,  2157,  4638,  1921,
        1828,  8043,   711,   784,   720,  5739,  4909,  4518,  1914,
        2168,  1555,  8024,  3791,  4909,  4518,  1316,  2768,   749,
        7484,  1462,  2157,  4638,  1921,  1828,  8043,   677,  3862,
         697,  1920,  4909,  4518,  8038,  4636,  2399,  1184,  4638,
         671,  1767,   100,  1920,  3124,  2424,   100,   680,   100,
        2207,  3124,  2424,   100,  4638,  1169,  2428,  4993,   751,
         511,  3341,  3975,  8038,   782,  4868,  1066,  1939,  8020,
         100,  8038,  8228,  8291, 11017,  8797,  9450,  8778,  8021,
         868,  5442,  8038,  3330,  1157,   122,   120,   127,   697,
         702,  4909,

In [68]:
len(x_train[1][0])

512

In [70]:
from transformers import TFBertForSequenceClassification, TFBertModel, BertConfig, TFBertForPreTraining

max_len = 512

def create_model():
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("bert-base-chinese")

    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
 
    start_logits0 = layers.Dense(1, name="start_logits0", use_bias=False)(embedding)
    #start_logits1 = (Activation('sigmoid'))(start_logits0)
    #start_logits = layers.Flatten()(start_logits0)

    #end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    #end_logits = layers.Flatten()(end_logits)

    #start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    #end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_logits0],
    )
    #loss = keras.losses.MSE(y_pred= model.outputs, y_true = y_val)
    #loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    
    model.layers[3].trainable = False  #make the bert layer not trainable
    model.compile(optimizer=optimizer, loss="mean_squared_error", metrics = tf.keras.metrics.MeanSquaredError(name='mse'))
    return model

mymodel = create_model()
mymodel.summary()

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_7 (TFBertModel)   TFBaseModelOutputWit 102267648   input_22[0][0]                   
                                                                 input_24[0][0]             

In [72]:
history = mymodel.fit(x_train, y_train, batch_size=64, epochs=3, validation_data=(x_val, y_val))

Epoch 1/3
125/125 [==============================] - 11818s 94s/step - loss: 1002122.5000 - mse: 1002122.5000 - val_loss: 811935.0625 - val_mse: 811935.0625
Epoch 2/3
125/125 [==============================] - 10984s 88s/step - loss: 1001569.8125 - mse: 1001569.8125 - val_loss: 811322.3125 - val_mse: 811322.3125
Epoch 3/3
125/125 [==============================] - 10839s 87s/step - loss: 1001024.8750 - mse: 1001024.8750 - val_loss: 810718.6875 - val_mse: 810718.6875


In [73]:
history.history

{'loss': [1002122.5, 1001569.8125, 1001024.875],
 'mse': [1002122.5, 1001569.8125, 1001024.875],
 'val_loss': [811935.0625, 811322.3125, 810718.6875],
 'val_mse': [811935.0625, 811322.3125, 810718.6875]}